In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from google.colab import drive
import os
import pickle
from scipy.signal import find_peaks, argrelextrema
from scipy.spatial.distance import cdist

In [ ]:
def RK4step(func, current_state, t, dt):
    k1 = func(t, current_state)
    k2 = func(t + 0.5 * dt, current_state + 0.5 * dt * k1)
    k3 = func(t + 0.5 * dt, current_state + 0.5 * dt * k2)
    k4 = func(t + dt, current_state + dt * k3)
    return current_state + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

# Lorenz system - Drive

\begin{align*}
\frac{dx}{dt} &= \sigma (y - x) \\
\frac{dy}{dt} &= x (\rho - z) - y \\
\frac{dz}{dt} &= xy - \beta z
\end{align*}

In [ ]:
class LorenzSystem:

    def __init__(self):
        self.sigma = 10.0
        self.rho = 28.0
        self.beta = 8.0 / 3.0
        self.state = np.array([1, 1, 1])

    def __call__(self, t, u):
        return np.array([
            self.sigma * (u[1] - u[0]),
            u[0] * (self.rho - u[2]) - u[1],
            u[0] * u[1] - self.beta * u[2]
        ])

    def reinitialise_ic(self):
        self.state = np.array([1, 1, 1])

    def randomise_ic(self):
        self.state = 2 * (np.random.rand(self.state.size) - 0.5)

# Reservoir System - Response
\begin{align*}
\frac{d}{dt}\mathbf{r}(t) = \gamma \left[ -r(t) + \tanh \left( M r(t) + \sigma W_{\text{in}} u(t) \right) \right]
\end{align*}

In [ ]:
class ReservoirComputer:
    def __init__(self, drive_system, reservoir_size, connectivity, spectral_radius, gamma, sigma):
        self.rng_M = np.random.default_rng(1911)
        self.rng_Win = np.random.default_rng(1912)
        self.rng_state = np.random.default_rng(1913)

        self.r = 2 * (self.rng_state.random(reservoir_size) - 0.5)
        self.u = drive_system
        self.spectral_radius = spectral_radius
        self.connectivity = connectivity
        self.reservoir_size = reservoir_size

        self.M, self.original_M, self.sparsification_mask = self.generate_matrix_and_mask(reservoir_size, connectivity, spectral_radius)
        self.Win = self.generate_Win(reservoir_size, self.u.state.size)

        self.sigma = sigma
        self.gamma = gamma
        self.res_update_func_listen = self.reservoir_update_equation
        self.res_update_func_pred = self.reservoir_update_equation_prediction

        self.dt = 0.01
        self.t = 0.0
        self.beta = 0.001
        self.q = lambda x: np.concatenate((np.array(x), np.array(x)**2))
        self.A = None
        self.b = None
        self.phi = None

    def generate_matrix_and_mask(self, reservoir_size, connectivity, spectral_radius):
        M = 2 * (self.rng_M.random((reservoir_size, reservoir_size)) - 0.5)
        sparsification_mask = self.rng_M.binomial(1, connectivity, size=(reservoir_size, reservoir_size))
        original_M = M.copy()
        sparsification_mask_copy = sparsification_mask.copy()
        M *= sparsification_mask
        M *= spectral_radius / max(np.abs(np.linalg.eigvals(M))).real
        return M, original_M, sparsification_mask_copy

    def generate_Win(self, reservoir_size, input_size):
        Win = np.zeros((reservoir_size, input_size))
        random_cols = self.rng_Win.integers(0, input_size, reservoir_size)
        random_values = self.rng_Win.uniform(-1, 1, reservoir_size)
        Win[np.arange(reservoir_size), random_cols] = random_values
        return Win

    def reservoir_update_equation(self, t, r):
        return self.gamma * (-r + np.tanh(self.M @ r + self.sigma * self.Win @ self.u.state))

    def reservoir_update_equation_prediction(self, t, r):
        return self.gamma * (-r + np.tanh(self.M @ r + self.sigma * self.Win @ self.phi(self.q(r))))

    def listen(self, t_listen):
        num_timesteps = int(t_listen / self.dt)

        for _ in range(num_timesteps):
            self.u.state = RK4step(self.u, self.u.state, self.t, self.dt)
            self.r = RK4step(self.res_update_func_listen, self.r, self.t, self.dt)
            self.t += self.dt

    def train(self, t_train):
        num_timesteps = int(t_train / self.dt)

        u_states = []
        symmetry_broken_r_states = []
        for _ in range(num_timesteps):
            self.u.state = RK4step(self.u, self.u.state, self.t, self.dt)
            u_states.append(self.u.state)
            self.r = RK4step(self.res_update_func_listen, self.r, self.t, self.dt)
            symmetry_broken_r_states.append(self.q(self.r))
            self.t += self.dt

        u_states = np.array(u_states)
        symmetry_broken_r_states = np.array(symmetry_broken_r_states)
        self.parameterise_phi(symmetry_broken_r_states, u_states)

    def predict(self, t_predict):
        num_timesteps = int(t_predict / self.dt)

        predictions = [self.phi(self.q(self.r))]
        for _ in range(num_timesteps):
            self.r = RK4step(self.res_update_func_pred, self.r, self.t, self.dt)
            prediction = self.phi(self.q(self.r))
            predictions.append(prediction)
            self.t += self.dt

        return np.array(predictions)

    def parameterise_phi(self, X, Y):
        n = X.shape[0]
        dims_x = X.shape[1]
        dims_y = Y.shape[1]
        A = np.linalg.inv(X.T @ X + self.beta * np.identity(dims_x)) @ X.T @ Y
        b = (1 / n) * np.sum(Y - X @ A, axis=0)
        self.A = A
        self.b = b
        self.phi = lambda x: self.A.T @ x + self.b
        #self.phi = lambda x: x @ self.A + self.b

    def set_spectral_radius(self, spectral_radius):
        if spectral_radius == 0:
            self.M = np.zeros_like(self.original_M)
        else:
            self.M = (self.original_M * self.sparsification_mask) * (spectral_radius / max(np.abs(np.linalg.eigvals(self.original_M * self.sparsification_mask))).real)
        self.spectral_radius = spectral_radius

    def time_ts(self, natural_time):
        return int(natural_time/self.dt)

    def reinitialise_same_RC(self):
        self.reinitialise_state()
        self.u.reinitialise_ic()
        self.A = None
        self.b = None
        self.phi = None

    def reinitialise_state(self):
        self.r = 2 * (self.rng_state.random(self.reservoir_size) - 0.5)
        self.t = 0.0

    def reinitialise_M(self):
        self.M, self.original_M, self.sparsification_mask = self.generate_matrix_and_mask(self.reservoir_size, self.connectivity, self.spectral_radius)

    def reinitialise_Win(self):
        self.Win = self.generate_Win(self.reservoir_size, self.u.state.size)

    def reinitialise_RC(self):
        self.reinitialise_M()
        self.reinitialise_Win()
        self.reinitialise_state()

## The following function `time_ts`converts the time parameter into timestep units

In [ ]:
def time_ts(natural_time):
        return int(natural_time/0.01)

# Object for RC simulation in bulk.

In [ ]:
class ReservoirComputer_BulkSimulation(ReservoirComputer):
    def __init__(self, drive_system, reservoir_size, connectivity, spectral_radius, gamma, sigma):
        super().__init__(drive_system, reservoir_size, connectivity, spectral_radius, gamma, sigma)

    def predict_100(self, t_predict):
        predictions_100 = []
        for _ in range(100):
            self.reinitialise_state()
            predictions = self.predict(t_predict)
            predictions_100.append(predictions)

        print("Predictions complete")
        return np.array(predictions_100)

    def simulate_100(self, t_listen, t_train, t_predict):
        self.rng_state = np.random.default_rng(1913)
        self.reinitialise_same_RC()

        self.listen(t_listen)
        self.train(t_train)
        trajectories = self.predict_100(t_predict)
        return np.array(trajectories)

# Perform 100 simulations of 100 trajectories for each spectral radius. 50 of the simulations randomise M, for 50 of them randomise Win times.

Structure of stored simulations, `tuple = (np.array(trajectories), np.array(seeds))`



In [ ]:
#constant parameters used across all simulations
lorenz = LorenzSystem()                 # U(t)
reservoir_size = 100                    # N
connectivity = 0.05                     # P
spectral_radius = 0.6                   # ρ
gamma = 10.0                            # γ
sigma = 0.2                             # σ

In [ ]:
params = (lorenz, reservoir_size, connectivity, spectral_radius, gamma, sigma)

In [ ]:
times = (100, 100, 100)

In [ ]:
# set of spectral radius values to be explored across
spectral_radii = np.linspace(0, 0.8, 17)
spectral_radii = [str(p) for p in spectral_radii]
spectral_radii.append('0.85')
spectral_radii.append('0.90')
spectral_radii.append('0.95')
spectral_radii.append('1.00')
spectral_radii.append('1.05')
spectral_radii.append('1.10')

spectral_radii

['0.0',
 '0.05',
 '0.1',
 '0.15000000000000002',
 '0.2',
 '0.25',
 '0.30000000000000004',
 '0.35000000000000003',
 '0.4',
 '0.45',
 '0.5',
 '0.55',
 '0.6000000000000001',
 '0.65',
 '0.7000000000000001',
 '0.75',
 '0.8',
 '0.85',
 '0.90',
 '0.95',
 '1.00',
 '1.05',
 '1.10']

In [ ]:
# set of parameters that depend on random initialisations to be considered across simulations
randMatrices = ['M', 'Win']


# Test for matrix reconstruction from seed.

In [ ]:
# method to regenerate specific topologies of M and Win from a seed stored. This ensures replicability.
def RC_regen_from_seed(params, seed, randomised_parameter):
  RC_B = ReservoirComputer_BulkSimulation(*params)
  if randMat == "M":
    RC_B.rng_M.bit_generator.state = seed
    RC_B.reinitialise_M()
  else:
    RC_B.rng_Win.bit_generator.state = seed
    RC_B.reinitialise_Win()
  return RC_B

In [ ]:
# test to reconstruct a RC from it's seed.
RC_B = ReservoirComputer_BulkSimulation(lorenz, reservoir_size, connectivity, spectral_radius, gamma, sigma)

seed_state_next = RC_B.rng_M.bit_generator.state
for i in range(5):
  seed_state_current = seed_state_next
  print(seed_state_current)
  seed_state_next = RC_B.rng_M.bit_generator.state
  RC_B.reinitialise_M()

M_1 = RC_B.M

RC_B2 = ReservoirComputer_BulkSimulation(lorenz, reservoir_size, connectivity, spectral_radius, gamma, sigma)
RC_B2.rng_M.bit_generator.state = seed_state_next
RC_B2.reinitialise_M()
M_2 = RC_B2.M

print((M_1 == M_2).all())

{'bit_generator': 'PCG64', 'state': {'state': 44492770237545670911816872350681932125, 'inc': 165062252236797752193301997657548364039}, 'has_uint32': 0, 'uinteger': 0}
{'bit_generator': 'PCG64', 'state': {'state': 44492770237545670911816872350681932125, 'inc': 165062252236797752193301997657548364039}, 'has_uint32': 0, 'uinteger': 0}
{'bit_generator': 'PCG64', 'state': {'state': 279476993459187764586166200538766514173, 'inc': 165062252236797752193301997657548364039}, 'has_uint32': 0, 'uinteger': 0}
{'bit_generator': 'PCG64', 'state': {'state': 199277921470031231992453321433218072221, 'inc': 165062252236797752193301997657548364039}, 'has_uint32': 0, 'uinteger': 0}
{'bit_generator': 'PCG64', 'state': {'state': 17214443678561910221255505079014397245, 'inc': 165062252236797752193301997657548364039}, 'has_uint32': 0, 'uinteger': 0}
True


# The juice

In [ ]:
# specify simulation being run. In this case it is 50 explorations of the state space of different RC for rho = 1.10, where each exploration has a different randomised initialisation of Win
p = spectral_radii[22]
randMat = randMatrices[1]
p, randMat

('1.10', 'Win')

In [ ]:
RC_B = ReservoirComputer_BulkSimulation(lorenz, reservoir_size, connectivity, spectral_radius, gamma, sigma)
RC_B.set_spectral_radius(float(p))

trajectories_list = []
seed_states_list = []

# Stores seed state used to generate the next Matrix in the analysis.
seed_state_next = RC_B.rng_M.bit_generator.state if randMat == "M" else RC_B.rng_Win.bit_generator.state

for i in range(50):

    # take the precalculated value of the current seed from its placeholder and calculate the seed state for the next matrix generation again
    seed_state_current = seed_state_next
    seed_state_next = RC_B.rng_M.bit_generator.state if randMat == 'M' else RC_B.rng_Win.bit_generator.state

    # perform exploration and append seed for replicability and results of exploration
    trajectories = RC_B.simulate_100(*times)
    trajectories_list.append(trajectories)
    seed_states_list.append(seed_state_current)

    # randomise parameter of interest for next exploration
    if randMat == "M":
        RC_B.reinitialise_M()
    else:
        RC_B.reinitialise_Win()

    print(f"\033[1mSimulation complete for ρ = {p}, randomised parameter = {randMat}, and realisation = {i} \033[0m")

result_tuple = (np.array(trajectories_list), np.array(seed_states_list))
bifurcation_analysis_data_piece = result_tuple

Predictions complete
Simulation complete for ρ = 1.10, randomised parameter = Win, and realisation = 0 
Predictions complete
Simulation complete for ρ = 1.10, randomised parameter = Win, and realisation = 1 
Predictions complete
Simulation complete for ρ = 1.10, randomised parameter = Win, and realisation = 2 
Predictions complete
Simulation complete for ρ = 1.10, randomised parameter = Win, and realisation = 3 
Predictions complete
Simulation complete for ρ = 1.10, randomised parameter = Win, and realisation = 4 
Predictions complete
Simulation complete for ρ = 1.10, randomised parameter = Win, and realisation = 5 
Predictions complete
Simulation complete for ρ = 1.10, randomised parameter = Win, and realisation = 6 
Predictions complete
Simulation complete for ρ = 1.10, randomised parameter = Win, and realisation = 7 
Predictions complete
Simulation complete for ρ = 1.10, randomised parameter = Win, and realisation = 8 
Predictions complete
Simulation complete for ρ = 1.10, randomise

In [ ]:
trajectories, seeds = bifurcation_analysis_data_piece

NameError: name 'bifurcation_analysis_data_piece' is not defined

In [ ]:
trajectories.shape

(50, 100, 10001, 3)

### Now to recreate reservoir used for analysis

`RC_B = ReservoirComputer_BulkSimulation(lorenz, reservoir_size, connectivity, spectral_radius, gamma, sigma)`
<br>
` RC_B.rng_M.bit_generator.state = seed_state`
<br>
`RC_B.reinitialise_M()`

 This leaves us with the Reservoir confiuration used to generate the trajectories associated with the seed_state from the tuple (trajectories, seed_state)

# Store and retrieval functions for cloud

In [ ]:
# function to store exploration data in drive for easy access from colab
from google.colab import drive
drive.mount('/content/drive')

def save_dictionary(dictionary, file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(file_path, 'wb') as file:
        pickle.dump(dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'Dictionary saved to {file_path}')

# def save_dictionary_to_drive(dictionary, file_path):
#     full_path = os.path.join('/content/drive/MyDrive', file_path)
#     directory = os.path.dirname(full_path)
#     if not os.path.exists(directory):
#         os.makedirs(directory)
#     with open(full_path, 'wb') as file:
#         pickle.dump(dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)
#     print(f'Dictionary saved to {full_path}')

def load_dictionary(file_path):
    with open(file_path, 'rb') as file:
        loaded_dictionary = pickle.load(file)
    print(f"Data loaded successfully from {file_path}")
    return loaded_dictionary

Mounted at /content/drive


In [ ]:
file_path = f'drive/My Drive/bifurcation_analysis_data/{p}_{randMat}.pkl'
# file_path = f'bifurcation_analysis_data/{p}_{randMat}.pkl'
file_path

'drive/My Drive/bifurcation_analysis_data/1.10_Win.pkl'

#### Store data we are working with.

In [ ]:
#save the 50 explorations under a relevant file path
save_dictionary(bifurcation_analysis_data_piece, file_path)

Dictionary saved to drive/My Drive/bifurcation_analysis_data/1.10_Win.pkl


#### Retrieve data we are working with.

In [ ]:
bifurcation_analysis_loaded = load_dictionary(file_path)

Data loaded successfully from drive/My Drive/bifurcation_analysis_data/0.05_M.pkl


In [ ]:
bifurcation_analysis_data_piece = bifurcation_analysis_loaded['M']

In [ ]:
trajectories_bulk, seed = bifurcation_analysis_loaded